In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np
from influxdb import *
#import Config as cg
#from influxdb import DataFrameClient
import datetime

INFLUX_DB_IP = '10.12.97.178'
INFLUX_DB_PORT = 8086
INFLUX_DB = 'demoDb'
TARGET_MEASUREMENT = 'EM_live'

con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
query = ('select "EM_TOTAL_Import_Energy(kWh)", "DeviceID", "time" from ' + TARGET_MEASUREMENT + ' where time > now() - 2d ')
df = pd.DataFrame(con_obj.query(query, chunked=True, chunk_size=10000).get_points())
df['time'] = df['time'].astype('datetime64[ns]')
df['time'] = df['time'] + datetime.timedelta(hours=5, minutes=30)

df.head(2)

df.tail(2)

# MAIN GRID

df1 = df[(df['time'] >= '2020-06-25 00:00:00.000000000') & (df['time'] < '2020-06-26 00:00:00.000000000')]
df1 = df1[df1['DeviceID'] == 'EM1']
df1['time'] = df1['time'].astype('datetime64[ns]')
df1 = df1.set_index('time')

print(df1.index.min())
print(df1.index.max())

x = df1.index.to_series().diff().dt.total_seconds().dropna()
x.describe()

# Gaps in data more than 1 Minute
a = x > 60
len(x[a])
x[a]

# Gaps in data more than 5 Minute
b = x > 300
len(x[b])
x[b]

print(x[b].values/3600)

# UPS 1

df2 = df[(df['time'] >= '2020-06-25 00:00:00.000000000') & (df['time'] < '2020-06-26 00:00:00.000000000')]
df2 = df2[df2['DeviceID'] == 'EM4']
df2['time'] = df2['time'].astype('datetime64[ns]')
df2 = df2.set_index('time')

print(df2.index.min())
print(df2.index.max())

y = df2.index.to_series().diff().dt.total_seconds().dropna()
y.describe()

# Gaps in data more than 1 Minute
c = y > 60
len(y[c])
y[c]

# Gaps in data more than 5 Minute
d = y > 300
len(y[d])
y[d]



# UPS 2

df3 = df[(df['time'] >= '2020-06-25 00:00:00.000000000') & (df['time'] < '2020-06-26 00:00:00.000000000')]
df3 = df3[df3['DeviceID'] == 'EM5']
df3['time'] = df3['time'].astype('datetime64[ns]')
df3 = df3.set_index('time')

print(df3.index.min())
print(df3.index.max())

z = df3.index.to_series().diff().dt.total_seconds().dropna()
z.describe()

# Gaps in data more than 1 Minute
e = z > 60
len(z[e])
z[e]

# Gaps in data more than 5 Minute
f = z > 300
len(z[f])
z[f]

#z[f].value()



In [ ]:
class half_an_hour_min_energy:  
    def difference(self,df3):
        diff_l=[]
        for g,i in df3.groupby(pd.Grouper(freq='30T',key='time'))['EM_TOTAL_Import_Energy(kWh)']:
            
            try:
                diff_l.append([g,i.iloc[-1]-i.iloc[0]])
            except:
                diff_l.append([g,np.nan])
#         x=pd.Series(diff_l)
        x=pd.DataFrame(diff_l)
        x.index=x[0]
        x=pd.Series(x[1])
#         print(x)
        return x
                
    def output(self, df3):
            y = df3.groupby('DeviceID').apply(self.difference).reset_index().T
            new_header = y.iloc[1]
            df = df[1:] #take the data less the header row
            df.columns = new_header 
            y = y.fillna(0)
            return y
            
            